In [3]:
from service.site_scraper import SiteScraper

gsp_scraper = SiteScraper(
    name="gsp",
    base_url="https://www.gsp.ro",
    traffic=3.6,
    time_selector="p.data-author",
    block_selector="div.news-item",
    link_selector="h2 a",
    title_strategy="text"
)

digisport_scraper = SiteScraper(
    name="digisport",
    base_url="https://www.digisport.ro",
    traffic=3.3,
    time_selector="cite",
    block_selector="article.article",
    link_selector="a.article-link, h3 a.widget-latest-list-item-link",
    title_strategy="attribute",
    title_attribute="title"
)

fanatik_scraper = SiteScraper(
    name="fanatik",
    base_url="https://www.fanatik.ro",
    traffic=2.9,  # Estimated, adjust as needed
    time_selector="span.date",  # Often used near article blocks
    block_selector="div.article",  # Container for each article
    link_selector="h3 a",  # Headline links
    title_strategy="text"
)

prosport_scraper = SiteScraper(
    name="prosport",
    base_url="https://www.prosport.ro",
    traffic=3.1,
    time_selector="span.date",
    block_selector="h2.article__title",  # Directly targets the title block
    link_selector="a",  # The anchor inside the h2
    title_strategy="text"
)

sites = [gsp_scraper, digisport_scraper, fanatik_scraper, prosport_scraper]
# sites = [prosport_scraper]
total_traffic = sum(site.traffic for site in sites)

In [4]:
for site in sites:
    site.compute_weight(total_traffic)
    site.scrape_recent_articles(minutes=360)
    # site.save_to_csv()
    # site.short_print()


In [5]:
for site in sites:
    site.save_to_csv()
    # site.short_print()

In [6]:
from service.story_clusterer import StoryClusterer

clusterer = StoryClusterer(sites, 360, 0.3, 0.2)
clusterer.cluster_stories()

clusterer.print_matched_clusters()


🧠 Cluster #1 — Score: 0.535

🧠 Cluster #2 — Score: 0.481

🧠 Cluster #3 — Score: 0.481

🧠 Cluster #4 — Score: 0.481

🧠 Cluster #5 — Score: 0.465

🧠 Cluster #6 — Score: 0.465

🧠 Cluster #7 — Score: 0.465

🔍 Matched Clusters Across Multiple Sites

🧠 Cluster #1 — Score: 0.535 — Sites: gsp, digisport
------------------------------------------------------------
📰 ”Nebunie” înainte de Go Ahead Eagles - FCSB: Suporterii voiau un stadion de 3 ori mai mare
🔗 https://www.digisport.ro/fotbal/europa-league/nebunie-inainte-de-go-ahead-eagles-fcsb-suporterii-voiau-un-stadion-de-3-ori-mai-mare-3827257
🔑 Keywords: întâlni, prima rundă, faza ligii, direct, ora 19:45
🏷️ Entities: FCSB, Go Ahead Eagles, Europa League, Digi Sport 1
------------------------------------------------------------
📰 Mihai Stoica, anunț important înainte de Go Ahead Eagles - FCSB: „Vor juca”
🔗 https://www.gsp.ro/international/europa-league/mihai-stoica-anunt-important-go-ahead-eagles-fcsb-846362.html
🔑 Keywords: președinte, Cons

In [7]:

top_stories = clusterer.score_clusters()

for i, story in enumerate(top_stories[:5], 1):
    print(f"\n🏆 Top {i} — Score: {story['score']}")
    for article in story["articles"]:
        print(f"- {article.site}: {article.title}")



🏆 Top 1 — Score: 0.535
- digisport: ”Nebunie” înainte de Go Ahead Eagles - FCSB: Suporterii voiau un stadion de 3 ori mai mare
- gsp: Mihai Stoica, anunț important înainte de Go Ahead Eagles - FCSB: „Vor juca”

🏆 Top 2 — Score: 0.481
- fanatik: Florentin Pera și Bogdan Burcea și-au dat demisia de la naționala de handbal! Haos cu două luni înainte de Campionatul Mondial
- digisport: Florentin Pera și-a dat demisia, cu doar două luni înainte de Campionatul Mondial!

🏆 Top 3 — Score: 0.481
- digisport: Câte pachete a vândut Universitatea Craiova cu aproape o lună înainte de debutul în Conference League
- fanatik: Nebunie în Bănie! Câte pachete pentru Conference League a vândut Universitatea Craiova. Exclusiv

🏆 Top 4 — Score: 0.481
- fanatik: Previziuni teribile pentru FCSB! Un supercomputer a calculat ce șanse are echipa lui Gigi Becali la calificarea în primăvara europeană
- digisport: Gigi Becali, imposibil de întors! Decizia patronului de la FCSB în cazul lui Daniel Bîrligea

🏆 Top 5